In [0]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns',None)

In [0]:
data_train = pd.read_csv('train_HK6lq50.csv')
data_test = pd.read_csv('test_wF0Ps6O.csv')

In [0]:
df_train = data_train.copy(deep = True)
df_test = data_test.copy(deep = True)

In [4]:
df_train.head()

,id,program_id,program_type,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,9389_150,Y_1,Y,136,150,offline,intermediate,9389,M,Matriculation,3,24.0,5,N,1.0,0
1,16523_44,T_1,T,131,44,offline,easy,16523,F,High School Diploma,4,26.0,2,N,3.0,1
2,13987_178,Z_2,Z,120,178,online,easy,13987,M,Matriculation,1,40.0,1,N,2.0,1
3,13158_32,T_2,T,117,32,offline,easy,13158,F,Matriculation,3,NaN,4,N,1.0,1
4,10591_84,V_3,V,131,84,offline,intermediate,10591,F,High School Diploma,1,42.0,2,N,4.0,1


## Eda

In [5]:
df_train.is_pass.value_counts()

1    50867
0    22280
Name: is_pass, dtype: int64

In [0]:
# for col in df_train.columns:
#     print(col)
#     print()
#     print(df_train[col].value_counts())
#     print('='*80)

In [0]:
# df_train.groupby('gender')['is_pass'].value_counts()    # drop gender

In [0]:
# df_train.groupby('is_handicapped')['is_pass'].value_counts(normalize = True)    # drop is_handicapped

In [0]:
# drop program_type as program id is related 

In [0]:
# df_train.groupby('city_tier')['is_pass'].value_counts(normalize = True)    

In [0]:
# df_train.groupby('trainee_engagement_rating')['is_pass'].value_counts(normalize = True) # cannot drop

In [0]:
def type_casting(data):
    df = data
    df['program_id'] = df['program_id'].astype('object')
    df['education'] = df['education'].astype('object')
    df['test_type'] = df['test_type'].astype('object')
    df['difficulty_level'] = df['difficulty_level'].astype('object')
    
    return(df)

In [0]:
def drop(data):
#     df = data.copy()
    df = data.drop(columns = ['is_handicapped','gender','program_type','id'])
    
    return df
    

In [0]:
def label_enc(dataset):
    from sklearn.preprocessing import LabelEncoder
    df1 = dataset.copy()
    df_categorical = df1.select_dtypes(include=['object'])

    # apply Label encoder to df_categorical

    le = LabelEncoder()
    df_categorical = df_categorical.apply(le.fit_transform)
    df_categorical = df_categorical.astype('object')

    # concat df_categorical with original df AFTER LABEL ENCODING
    df1 = df1.drop(df_categorical.columns, axis=1)
    df1 = pd.concat([df1, df_categorical], axis=1)
    
    df_categorical =0
    return df1

In [0]:
def impute(data):
    from sklearn.impute import SimpleImputer
    
    si = SimpleImputer()
    array = si.fit_transform(data)
    
    return array
    

In [0]:
def standardize(dataset, X_test = None ,test = False):
    from sklearn.preprocessing import StandardScaler
    df1 = dataset.copy()
    df_num = df1.select_dtypes(include=['int64','float64','int32','float32'])
    df_num = pd.DataFrame(impute(df_num), columns = df_num.columns,index = df1.index)

    se = StandardScaler()
    df_scaled = pd.DataFrame(se.fit_transform(df_num),columns = df_num.columns, index = df_num.index)
    
    df_new = df1.drop(df_num.columns,axis =1)
    df_new = pd.concat([df_scaled,df_new],axis =1)
    
    if test:
        df1 = X_test.copy()
        df_test_num = X_test.select_dtypes(include=['int64','float64','int32','float32'])
        
        df_scaled = pd.DataFrame(se.transform(df_test_num),columns = df_test_num.columns, index = df_test_num.index)
        
        df_new_ = df1.drop(df_test_num.columns,axis =1)
        df_new_ = pd.concat([df_new_,df_scaled],axis =1)
    else:
        df_new_ = 0
    
#     df_num ,df1,df_scaled = 0    
    return df_new,df_new_

In [0]:
# df = drop(df_train)
# df = type_casting(df)
# df = label_enc(df)
# df_tr,_ = standardize(df)
# # df_tr

## Split

In [0]:
df = drop(df_train)
df = type_casting(df)
df = label_enc(df)

In [0]:
X = df.drop('is_pass',axis = 1)
y = df.is_pass

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X,y, random_state = 6, stratify = y)

In [0]:
X_train1,_ = standardize(X_train)

In [0]:
x_train,x_test = standardize(X_train,X_test, test = True)

####test

In [0]:
df_test_after = drop(df_test)
df_test_after = type_casting(df_test_after)
df_test_final = label_enc(df_test_after)

In [71]:
df_test_final.shape

(31349, 11)

In [72]:
df_test_final.isnull().sum()

program_duration                 0
test_id                          0
trainee_id                       0
city_tier                        0
age                          11791
total_programs_enrolled          0
trainee_engagement_rating       31
program_id                       0
test_type                        0
difficulty_level                 0
education                        0
dtype: int64

## ML models

In [0]:
def cls(model):
    from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
    
    model = model()
    model.fit(X_train1, y_train)

    y_predict = model.predict(X_test1)
    print(confusion_matrix(y_test,y_predict)) 
    print(accuracy_score(y_test,y_predict))

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [0]:
X_test1,_ = standardize(X_test)

In [0]:
cls(DecisionTreeClassifier)

In [0]:
cls(LogisticRegression)

In [0]:
cls(AdaBoostClassifier)

In [0]:
cls(RandomForestClassifier)

In [0]:
cls(GradientBoostingClassifier)

In [0]:
cls(XGBClassifier)

## PyCaret

In [0]:
df.info()

In [34]:
df.columns

Index(['program_duration', 'test_id', 'trainee_id', 'city_tier', 'age',
       'total_programs_enrolled', 'trainee_engagement_rating', 'is_pass',
       'program_id', 'test_type', 'difficulty_level', 'education'],
      dtype='object')

In [13]:
pip install pycaret

     |████████████████████████████████| 194kB 2.4MB/s 
     |████████████████████████████████| 266kB 4.7MB/s 
     |████████████████████████████████| 81kB 4.3MB/s 
     |████████████████████████████████| 133kB 6.2MB/s 
     |████████████████████████████████| 61kB 3.7MB/s 
     |████████████████████████████████| 3.0MB 7.0MB/s 
     |████████████████████████████████| 7.0MB 24.0MB/s 
     |████████████████████████████████| 1.6MB 42.5MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 389kB 53.8MB/s 
     |████████████████████████████████| 1.2MB 59.6MB/s 
     |████████████████████████████████| 63.9MB 64kB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 614kB 44.5MB/s 
     |████████████████████████████████| 235kB 34.1MB/s 
     |████████████████████████████████| 6.1MB 46.9MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 552kB 46.0MB/s 
     

In [0]:
from pycaret.classification import *

###rgd

In [0]:
setup_data = setup(df, target='is_pass', session_id= 5,normalize=True, profile = True,
              numeric_features = ['program_duration','test_id', 'trainee_id', 'city_tier', 'age','total_programs_enrolled', 'trainee_engagement_rating'],
              categorical_features = ['program_id', 'test_type', 'difficulty_level', 'education'])

In [0]:
s

In [38]:
lr = create_model('lr',fold = 10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.7159,0.6987,0.9253,0.7348,0.8191,0.1956
1,0.7151,0.7083,0.9245,0.7345,0.8186,0.1938
2,0.7211,0.7013,0.9298,0.7376,0.8226,0.2092
3,0.7158,0.7014,0.9264,0.7344,0.8193,0.1937
4,0.7162,0.7115,0.9228,0.7361,0.8189,0.1997
5,0.7203,0.7085,0.9323,0.7360,0.8226,0.2032
6,0.7086,0.6933,0.9281,0.7278,0.8158,0.1649
7,0.7170,0.7079,0.9270,0.7351,0.8200,0.1979
8,0.7119,0.7039,0.9219,0.7328,0.8165,0.1853
9,0.7242,0.7175,0.9404,0.7361,0.8259,0.2083


In [37]:
dt = create_model('dt')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.6891,0.6324,0.7776,0.7758,0.7767,0.2652
1,0.6860,0.6325,0.7694,0.7769,0.7731,0.2633
2,0.6979,0.6470,0.7770,0.7861,0.7815,0.2919
3,0.6996,0.6477,0.7804,0.7861,0.7833,0.2941
4,0.6850,0.6329,0.7661,0.7777,0.7718,0.2632
5,0.6855,0.6293,0.7731,0.7744,0.7737,0.2584
6,0.6920,0.6361,0.7792,0.7781,0.7787,0.2724
7,0.6818,0.6302,0.7624,0.7761,0.7692,0.2575
8,0.7082,0.6598,0.7837,0.7940,0.7888,0.3170
9,0.6961,0.6410,0.7820,0.7811,0.7816,0.2822


In [39]:
tuned_dt = tune_model('dt', fold=5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.7131,0.6968,0.9179,0.7353,0.8165,0.1944
1,0.7159,0.6918,0.8968,0.7460,0.8145,0.2298
2,0.7146,0.7058,0.8825,0.7509,0.8114,0.2411
3,0.7116,0.6931,0.9054,0.7388,0.8137,0.2042
4,0.7178,0.6995,0.9086,0.7429,0.8174,0.2226
Mean,0.7146,0.6974,0.9022,0.7428,0.8147,0.2184
SD,0.0021,0.0050,0.0120,0.0054,0.0021,0.0170


In [40]:
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.7030,0.7009,0.7967,0.7807,0.7886,0.2896
1,0.7153,0.7239,0.8048,0.7897,0.7972,0.3196
2,0.7160,0.7232,0.8068,0.7895,0.7981,0.3200
3,0.7119,0.7244,0.8048,0.7861,0.7953,0.3092
4,0.7057,0.7142,0.8060,0.7786,0.7921,0.2891
5,0.7178,0.7263,0.8135,0.7877,0.8004,0.3193
6,0.7129,0.7124,0.8124,0.7829,0.7974,0.3056
7,0.7053,0.7067,0.8081,0.7769,0.7922,0.2860
8,0.7105,0.7258,0.8062,0.7837,0.7948,0.3041
9,0.7182,0.7228,0.8208,0.7840,0.8020,0.3142


In [41]:
rf.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=5, verbose=0,
                       warm_start=False)>

In [42]:
rf.n_features_

39

In [43]:
rf.feature_importances_

array([0.01440632, 0.17070111, 0.31141709, 0.06900554, 0.16038247,
       0.07150908, 0.07472877, 0.00105689, 0.00124127, 0.00255476,
       0.00524251, 0.00176464, 0.00114779, 0.00111598, 0.00219379,
       0.00203071, 0.00270674, 0.00205182, 0.00081734, 0.00472491,
       0.00130411, 0.00113707, 0.00171488, 0.00319881, 0.00194843,
       0.00388892, 0.00413858, 0.00196831, 0.00215838, 0.01758395,
       0.00873493, 0.00470559, 0.00521354, 0.00603349, 0.00981989,
       0.01033015, 0.00174547, 0.01152827, 0.00204768])

In [44]:
tuned_rf = tune_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.7405,0.7433,0.8961,0.7689,0.8276,0.3147
1,0.7393,0.7608,0.8936,0.7690,0.8266,0.3133
2,0.7414,0.7540,0.8978,0.7691,0.8285,0.3160
3,0.7367,0.7579,0.8983,0.7644,0.8260,0.2996
4,0.7330,0.7573,0.8947,0.7626,0.8234,0.2907
5,0.7430,0.7600,0.9099,0.7651,0.8312,0.3092
6,0.7340,0.7509,0.9003,0.7609,0.8248,0.2887
7,0.7369,0.7531,0.8992,0.7641,0.8262,0.2998
8,0.7410,0.7699,0.8958,0.7695,0.8279,0.3169
9,0.7412,0.7669,0.9076,0.7644,0.8298,0.3059


In [60]:
print(tuned_rf)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=60, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=5, verbose=0,
                       warm_start=False)


In [51]:
lbgm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.7333,0.7364,0.9101,0.7560,0.8259,0.2757
1,0.7327,0.7462,0.9155,0.7532,0.8265,0.2679
2,0.7365,0.7437,0.9233,0.7534,0.8298,0.2726
3,0.7312,0.7436,0.9118,0.7535,0.8252,0.2666
4,0.7240,0.7516,0.9037,0.7505,0.8200,0.2501
5,0.7428,0.7515,0.9275,0.7572,0.8338,0.2902
6,0.7289,0.7388,0.9228,0.7469,0.8256,0.2465
7,0.7297,0.7441,0.9166,0.7501,0.8250,0.2562
8,0.7354,0.7525,0.9149,0.7559,0.8278,0.2780
9,0.7426,0.7542,0.9244,0.7583,0.8332,0.2932


In [54]:
lgbm_tuned = tune_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.7356,0.7524,0.8978,0.7636,0.8252,0.2966
1,0.7397,0.7615,0.9012,0.7659,0.8280,0.3071
2,0.7406,0.7571,0.9045,0.7653,0.8291,0.3066
3,0.7383,0.7627,0.8992,0.7655,0.8270,0.3040
4,0.7410,0.7711,0.8995,0.7679,0.8285,0.3130
5,0.7396,0.7582,0.9076,0.7630,0.8290,0.3001
6,0.7387,0.7604,0.9065,0.7625,0.8283,0.2983
7,0.7346,0.7539,0.9003,0.7615,0.8251,0.2907
8,0.7361,0.7645,0.8952,0.7652,0.8251,0.3011
9,0.7482,0.7655,0.9059,0.7717,0.8334,0.3313


In [55]:
nb_tuned = tune_model('nb')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.6973,0.6665,0.8644,0.7426,0.7989,0.2016
1,0.6999,0.6755,0.8646,0.7448,0.8003,0.2100
2,0.6980,0.6675,0.8632,0.7438,0.7991,0.2051
3,0.6975,0.6716,0.8691,0.7408,0.7998,0.1963
4,0.6984,0.6762,0.8585,0.7462,0.7984,0.2118
5,0.7102,0.6821,0.8773,0.7490,0.8081,0.2311
6,0.6947,0.6594,0.8697,0.7380,0.7985,0.1865
7,0.7078,0.6796,0.8719,0.7490,0.8058,0.2294
8,0.6941,0.6769,0.8542,0.7439,0.7952,0.2023
9,0.7078,0.6880,0.8787,0.7462,0.8070,0.2218


In [56]:
mlp = create_model('mlp')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.7135,0.7000,0.8860,0.7483,0.8114,0.2334
1,0.7141,0.7167,0.8809,0.7510,0.8108,0.2410
2,0.7145,0.7106,0.8677,0.7572,0.8087,0.2561
3,0.7199,0.7164,0.8630,0.7646,0.8108,0.2792
4,0.7197,0.7246,0.8655,0.7632,0.8112,0.2760
5,0.7258,0.7232,0.8840,0.7606,0.8177,0.2773
6,0.7176,0.7108,0.8899,0.7504,0.8142,0.2432
7,0.7199,0.7130,0.8607,0.7656,0.8104,0.2818
8,0.7129,0.7151,0.8573,0.7603,0.8059,0.2621
9,0.7244,0.7268,0.8657,0.7676,0.8137,0.2915


In [57]:
cat = tune_model('catboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.7325,0.7312,0.9124,0.7544,0.8259,0.2705
1,0.7338,0.7491,0.9141,0.7549,0.8269,0.2735
2,0.7340,0.7443,0.9166,0.7540,0.8274,0.2710
3,0.7271,0.7403,0.9104,0.7505,0.8227,0.2537
4,0.7318,0.7521,0.9135,0.7534,0.8257,0.2668
5,0.7377,0.7531,0.9239,0.7542,0.8305,0.2762
6,0.7283,0.7383,0.9202,0.7474,0.8249,0.2472
7,0.7303,0.7431,0.9126,0.7523,0.8247,0.2626
8,0.7283,0.7495,0.9093,0.7519,0.8231,0.2595
9,0.7414,0.7539,0.9258,0.7567,0.8327,0.2876


In [59]:
svm = tune_model('svm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.7003,0.0,0.9832,0.7036,0.8202,0.0505
1,0.6962,0.0,0.9823,0.7009,0.8181,0.0340
2,0.7018,0.0,0.9834,0.7046,0.8210,0.0560
3,0.6982,0.0,0.9809,0.7028,0.8189,0.0449
4,0.7002,0.0,0.9815,0.7041,0.8199,0.0525
5,0.7043,0.0,0.9834,0.7065,0.8223,0.0670
6,0.6945,0.0,0.9817,0.6998,0.8172,0.0281
7,0.7002,0.0,0.9829,0.7036,0.8201,0.0510
8,0.6998,0.0,0.9820,0.7036,0.8198,0.0506
9,0.7014,0.0,0.9823,0.7046,0.8206,0.0569


In [61]:
compare_models(fold = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,CatBoost Classifier,0.734000,0.750200,0.911300,0.756200,0.826500,0.276900
1,Light Gradient Boosting Machine,0.730500,0.742600,0.915900,0.751200,0.825400,0.259700
2,Gradient Boosting Classifier,0.722900,0.721100,0.931200,0.738500,0.823700,0.214300
3,Extreme Gradient Boosting,0.722200,0.719900,0.933300,0.737200,0.823700,0.209400
4,Ada Boost Classifier,0.716700,0.705800,0.926700,0.735100,0.819800,0.197000
5,Linear Discriminant Analysis,0.716700,0.710700,0.919600,0.737700,0.818700,0.205700
6,Ridge Classifier,0.716400,0.000000,0.940100,0.729900,0.821700,0.178700
7,Logistic Regression,0.716200,0.704300,0.927700,0.734200,0.819700,0.193900
8,Extra Trees Classifier,0.706500,0.708000,0.824600,0.769700,0.796200,0.274100
9,Random Forest Classifier,0.703500,0.706600,0.801600,0.778600,0.789900,0.286600


In [0]:
df

In [63]:
df.is_pass.value_counts()

1    50867
0    22280
Name: is_pass, dtype: int64

In [0]:
predict = predict_model(cat, data = df_test_final)

In [74]:
predict

,program_duration,test_id,trainee_id,city_tier,age,total_programs_enrolled,trainee_engagement_rating,program_id,test_type,difficulty_level,education,Label,Score
0,131,45,1626,3,46.0,2,4.0,2,0,2,3,1,0.7552
1,135,130,11020,3,NaN,4,4.0,17,1,0,0,1,0.8645
2,120,146,12652,3,NaN,2,3.0,16,1,0,3,1,0.6458
3,122,72,7038,1,NaN,2,2.0,11,0,3,1,0,0.4105
4,122,71,888,3,NaN,2,2.0,11,0,2,3,1,0.5212
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31344,134,22,18820,2,22.0,2,1.0,4,0,2,4,0,0.3393
31345,133,96,1641,1,41.0,2,3.0,8,0,0,3,1,0.8524
31346,136,157,7010,2,NaN,3,4.0,15,1,0,0,1,0.8024
31347,136,179,14895,3,NaN,1,1.0,19,0,0,3,0,0.4104


In [0]:
predict.Label.value_counts()

In [0]:
predict_rgd_cat_py = predict.Label

In [0]:
predict_rgd_cat_py.to_csv('test_predict_rgd_cat_py.csv')

###py

In [0]:
setup_data = setup(df, target='is_pass', session_id= 5,normalize=True)

In [85]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,CatBoost Classifier,0.736900,0.755400,0.913600,0.757800,0.828400,0.284500
1,Light Gradient Boosting Machine,0.732500,0.746800,0.917500,0.752300,0.826700,0.264900
2,Gradient Boosting Classifier,0.722500,0.720000,0.930600,0.738400,0.823400,0.213400
3,Extreme Gradient Boosting,0.721500,0.719100,0.931800,0.737200,0.823100,0.208600
4,Ada Boost Classifier,0.716600,0.704900,0.931800,0.733100,0.820600,0.190200
5,Linear Discriminant Analysis,0.716200,0.694200,0.928900,0.733800,0.819900,0.192300
6,Logistic Regression,0.715700,0.695200,0.933400,0.731700,0.820300,0.184700
7,Ridge Classifier,0.715000,0.000000,0.942600,0.727900,0.821400,0.170200
8,Random Forest Classifier,0.710800,0.718200,0.802200,0.786300,0.794200,0.308300
9,Extra Trees Classifier,0.709000,0.716200,0.813200,0.778300,0.795300,0.292600


In [0]:
setup_data = setup(df, target='is_pass', session_id= 5,normalize=False,silent = True)

In [87]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,CatBoost Classifier,0.736900,0.755400,0.913600,0.757800,0.828400,0.284500
1,Light Gradient Boosting Machine,0.731600,0.746500,0.916900,0.751700,0.826100,0.262500
2,Gradient Boosting Classifier,0.722500,0.720000,0.930600,0.738400,0.823400,0.213400
3,Extreme Gradient Boosting,0.721500,0.719100,0.931800,0.737200,0.823100,0.208600
4,Ada Boost Classifier,0.716600,0.704900,0.931800,0.733100,0.820600,0.190200
5,Linear Discriminant Analysis,0.716200,0.694200,0.928900,0.733800,0.819900,0.192300
6,Ridge Classifier,0.715000,0.000000,0.942600,0.727900,0.821400,0.170200
7,Random Forest Classifier,0.709300,0.716500,0.803100,0.784100,0.793500,0.303000
8,Extra Trees Classifier,0.708700,0.716300,0.814100,0.777500,0.795400,0.291100
9,Logistic Regression,0.706300,0.633600,0.962900,0.714900,0.820100,0.104100


In [97]:
setup_data1 = setup(df, target = 'is_pass', session_id = 5, silent = True, normalize = True,transformation=True,
                    remove_outliers= True, feature_selection=True, pca =True, remove_multicollinearity= True)

 
Setup Succesfully Completed!


,Description,Value
0,session_id,5
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(73147, 12)"
4,Missing Values,True
5,Numeric Features,4
6,Categorical Features,7
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [98]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,CatBoost Classifier,0.731700,0.736100,0.910000,0.755600,0.825600,0.264900
1,Light Gradient Boosting Machine,0.727300,0.728400,0.918500,0.748200,0.824600,0.239300
2,Gradient Boosting Classifier,0.720600,0.708600,0.933300,0.736700,0.823400,0.196300
3,Extreme Gradient Boosting,0.720300,0.709200,0.934600,0.736000,0.823500,0.193500
4,Logistic Regression,0.715900,0.686400,0.945600,0.728400,0.822900,0.162000
5,Linear Discriminant Analysis,0.715600,0.684900,0.940500,0.730000,0.822000,0.167900
6,Ridge Classifier,0.714700,0.000000,0.957100,0.723500,0.824000,0.141300
7,Ada Boost Classifier,0.714600,0.690300,0.920000,0.736700,0.818200,0.190700
8,Random Forest Classifier,0.714300,0.716100,0.807500,0.788400,0.797800,0.311300
9,Extra Trees Classifier,0.709900,0.713400,0.798900,0.788400,0.793600,0.305700


In [0]:
setup_data1 = setup(df_train, target = 'is_pass', session_id = 5, silent = True, ignore_features = ['id'])

In [89]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,CatBoost Classifier,0.738500,0.758700,0.914300,0.759000,0.829400,0.289500
1,Light Gradient Boosting Machine,0.732800,0.748000,0.918300,0.752200,0.827000,0.265300
2,Gradient Boosting Classifier,0.723100,0.721100,0.931500,0.738600,0.823900,0.214700
3,Extreme Gradient Boosting,0.723000,0.719700,0.934600,0.737300,0.824300,0.210500
4,Linear Discriminant Analysis,0.719900,0.713900,0.916700,0.741500,0.819900,0.220600
5,Ridge Classifier,0.719600,0.000000,0.933400,0.734900,0.822300,0.199300
6,Ada Boost Classifier,0.718200,0.706800,0.930300,0.735000,0.821200,0.198100
7,Random Forest Classifier,0.716600,0.726600,0.810500,0.788000,0.799100,0.318600
8,Extra Trees Classifier,0.715100,0.725000,0.819500,0.781500,0.800000,0.305800
9,Logistic Regression,0.714000,0.673600,0.939900,0.728500,0.820500,0.167500


In [92]:
setup_data1 = setup(df_train, target = 'is_pass', session_id = 5, silent = True, ignore_features = ['id'], normalize = True,transformation=True)

 
Setup Succesfully Completed!


,Description,Value
0,session_id,5
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(73147, 16)"
4,Missing Values,True
5,Numeric Features,4
6,Categorical Features,11
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [93]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,CatBoost Classifier,0.738500,0.758600,0.914300,0.759000,0.829400,0.289500
1,Light Gradient Boosting Machine,0.732300,0.747700,0.918200,0.751800,0.826700,0.263400
2,Gradient Boosting Classifier,0.723100,0.721100,0.931500,0.738600,0.823900,0.214700
3,Extreme Gradient Boosting,0.723000,0.719700,0.934600,0.737300,0.824300,0.210500
4,Logistic Regression,0.720300,0.711200,0.925900,0.738300,0.821600,0.211200
5,Linear Discriminant Analysis,0.720000,0.713500,0.916700,0.741700,0.820000,0.221200
6,Ridge Classifier,0.719300,0.000000,0.933300,0.734700,0.822200,0.198400
7,Ada Boost Classifier,0.718200,0.706800,0.930300,0.735000,0.821200,0.198100
8,Random Forest Classifier,0.715600,0.726300,0.807300,0.788700,0.797900,0.318200
9,Extra Trees Classifier,0.715500,0.725600,0.820600,0.781300,0.800500,0.306100


In [99]:
setup_data1 = setup(df_train, target = 'is_pass', session_id = 5, silent = True, ignore_features = ['id'], normalize = True,transformation=True,
                    remove_outliers= True, feature_selection=True, pca =True, remove_multicollinearity= True)

 
Setup Succesfully Completed!


,Description,Value
0,session_id,5
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(73147, 16)"
4,Missing Values,True
5,Numeric Features,4
6,Categorical Features,11
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [100]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,CatBoost Classifier,0.733100,0.743100,0.904400,0.759500,0.825600,0.274600
1,Light Gradient Boosting Machine,0.728200,0.734500,0.913900,0.751000,0.824500,0.246400
2,Gradient Boosting Classifier,0.722300,0.719400,0.937700,0.736800,0.825200,0.195700
3,Extreme Gradient Boosting,0.722100,0.718700,0.940700,0.735400,0.825500,0.190600
4,Logistic Regression,0.719400,0.702400,0.929900,0.737200,0.822400,0.194600
5,Linear Discriminant Analysis,0.719200,0.701800,0.924500,0.739100,0.821500,0.200600
6,Ridge Classifier,0.718600,0.000000,0.939700,0.732900,0.823500,0.178600
7,Ada Boost Classifier,0.717400,0.699700,0.916200,0.740800,0.819200,0.204500
8,Extra Trees Classifier,0.716300,0.723000,0.817200,0.785500,0.801000,0.307700
9,K Neighbors Classifier,0.711800,0.695600,0.849600,0.764300,0.804700,0.260900


In [15]:
df.columns

Index(['program_duration', 'test_id', 'trainee_id', 'city_tier', 'age',
       'total_programs_enrolled', 'trainee_engagement_rating', 'is_pass',
       'program_id', 'test_type', 'difficulty_level', 'education'],
      dtype='object')

In [0]:
from imblearn.over_sampling import SMOTE

In [0]:
X_os = df.drop(columns='is_pass')
y_os = df.is_pass

In [0]:
def impute(data):
    from sklearn.impute import SimpleImputer
    
    si = SimpleImputer()
    array = si.fit_transform(data)
    
    return array

In [0]:
X_os = pd.DataFrame(impute(X_os), columns = X_os.columns,index = X_os.index)


In [0]:
os = SMOTE(random_state=0)
columns = X_os.columns

X_train_os,y_train_os=os.fit_sample(X_os, y_os)

X_train_os = pd.DataFrame(data=X_train_os,columns=columns )


In [0]:
df_os = X_train_os.copy(deep = True)
df_os['is_pass'] = y_train_os

In [121]:
setup_data1 = setup(df_os, target = 'is_pass', session_id = 5, silent = True, normalize = True,transformation=True,
                    remove_outliers= True, feature_selection=True, pca =True, remove_multicollinearity= True)

 
Setup Succesfully Completed!


,Description,Value
0,session_id,5
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(101734, 12)"
4,Missing Values,False
5,Numeric Features,11
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [123]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,Extra Trees Classifier,0.776000,0.819000,0.754300,0.784900,0.769300,0.551800
1,Random Forest Classifier,0.763000,0.829800,0.748100,0.767200,0.757500,0.525800
2,CatBoost Classifier,0.736700,0.818500,0.752100,0.726000,0.738800,0.473600
3,Decision Tree Classifier,0.734600,0.749800,0.725900,0.734700,0.730200,0.469000
4,K Neighbors Classifier,0.720000,0.791700,0.745400,0.705500,0.724900,0.440300
5,Light Gradient Boosting Machine,0.717700,0.797300,0.730800,0.708300,0.719300,0.435600
6,Gradient Boosting Classifier,0.684200,0.749100,0.685700,0.679300,0.682500,0.368400
7,Extreme Gradient Boosting,0.683600,0.747900,0.684100,0.679100,0.681600,0.367200
8,Ada Boost Classifier,0.650700,0.710500,0.634700,0.651000,0.642700,0.301200
9,Quadratic Discriminant Analysis,0.641200,0.692100,0.666400,0.630000,0.647700,0.282700


In [2]:
setup_data1 = setup(df_os, target = 'is_pass', session_id = 5, silent = False, normalize = True,transformation=True,
                    remove_outliers= True, feature_selection=True, pca =True, remove_multicollinearity= True,
                    numeric_features = ['program_duration','test_id', 'trainee_id', 'city_tier', 'age','total_programs_enrolled', 'trainee_engagement_rating'],
              categorical_features = ['program_id', 'test_type', 'difficulty_level', 'education'])

NameError: ignored

In [0]:
compare_models()

In [3]:
df

NameError: ignored